# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [ ]:
import json

In [ ]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [ ]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [ ]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [ ]:
print('notice that the instructor and code are tuples now')
courses[1246]

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

# a) How many faculty taught COSI courses last year?

In [ ]:
faculty = len({c['instructor'] for c in courses if c['subject'] == 'COSI'})
print(f"There were {faculty} faculties teaching COSI courses last year.")

# b) what is the total number of students taking COSI courses last year?

In [ ]:
# {c['enrolled'] for c in courses}
enroll = sum(c['enrolled'] for c in courses if c['subject'] == 'COSI')
print(f"There were {enroll} students taking COSI courses last year.")

# c) what was the median size of a COSI course last year (counting only those courses with at least 10 students)

In [ ]:
# size = {c for c in courses if c['subject'] == 'COSI' and c['enrolled'] >=10}
import statistics
student_size = [c['enrolled'] for c in courses if c['subject'] == 'COSI' and c['enrolled'] >= 10]
median = statistics.median(student_size)
print(f"The median size of a COSI course last year is {median} students.")

# d) create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.


In [ ]:
course_num = dict.fromkeys(set(c['subject'] for c in courses), 0)
for c in courses:
    course_num[c['subject']] += c['enrolled']

result = sorted(course_num.items(), key=lambda item: item[1], reverse=True)
print([(i[1], i[0]) for i in result[:10]])

# e) do the same as in (d) but print the top 10 subjects in terms of number of courses offered


In [ ]:
from collections import Counter
subject_count = Counter([course['subject'] for course in courses])
result = sorted(subject_count.items(), key=lambda item: item[1], reverse=True)
print([i[0] for i in result[:10]])

# f) do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject


In [ ]:
course_num = dict()
for c in courses:
    if c['subject'] not in course_num:
        course_num[c['subject']] = set()
    course_num[c['subject']].add(c['instructor'][0] + ' ' + c['instructor'][1])

result = sorted(course_num.items(), key=lambda item: len(item[1]), reverse=True)
print([s[0] for s in result[:10]])

# g) list the top 20 faculty in terms of number of students they taught


In [ ]:
instructors = {}
for course in courses:
    instructor = course['instructor'][0] + ' ' + course['instructor'][1]
    if instructor not in instructors:
        instructors[instructor] = []
    else:
        instructors[instructor].append(course['enrolled'])

for instructor in instructors:
    instructors[instructor] = sum(instructors[instructor])
    
sort_instructors = sorted(instructors.items(), key=lambda x: x[1], reverse=True)
[i[0] for i in sort_instructors[:20]]

# h) list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)


In [ ]:
course_enrollments = {}
for course in courses:
    course_name = course['subject'] + ' ' + course['coursenum'] + ' ' + course['name']
    if course_name not in course_enrollments:
        course_enrollments[course_name] = []
    else:
        course_enrollments[course_name].append(course['enrolled'])

for course in course_enrollments:
    course_enrollments[course] = sum(course_enrollments[course])
    
sort_course_enrollments = sorted(course_enrollments.items(), key=lambda x: x[1], reverse=True)
[i[0] for i in sort_course_enrollments[:20]]

# i) Create your own interesting question (each team member creates their own) and use Python to answer that question.

Alicia Sheng

New change

Question: What are the top 20 courses with the highest enrollment rate (enrolled / limit)?

In [ ]:
enrollment_rate = [(course['subject'] + ' ' + course['coursenum'] + ' ' + course['name'], course['enrolled'] / course['limit']) for course in courses if course['limit'] != None and course['limit'] != 0]
enrollment_rate.sort(key=lambda x: x[1], reverse=True)
[i[0] for i in enrollment_rate[:20]]

Francisco Liu

Question: Among all independent study courses, what are the top 5 courses that have the most enrolled students? Display them in a format of a list of tuple (course_name, enrolled student). 

In [ ]:
all_independent_study = dict()

for course in courses:
    if course['independent_study'] == True and course['enrolled']!=0:
        all_independent_study[course['code'][0] + " " + course['code'][1]] = course['enrolled']

result = sorted(all_independent_study.items(), key=lambda item: item[1], reverse=True)
print([(i[0], i[1]) for i in result[:5]])


Gordon Dou

Question: What are the top 10 classes that have more enrollments than limits?

In [ ]:
enrollment_exceed = [course for course in courses if course['enrolled'] is not None and course['limit'] is not None and int(course['enrolled']) > int(course['limit'])]
enrollment_exceed.sort(key=lambda x: x['enrolled'] - x['limit'], reverse=True)
[i['name'] for i in enrollment_exceed[:10]]

Michael Li

Question: What is the the average size of a COSI course

In [ ]:
import statistics
student_size = {c['enrolled'] for c in courses if c['subject'] == 'COSI' and c['enrolled'] >=10}
mean = int(statistics.mean(student_size))
num = print(f"The average size of a COSI course last year was {mean} students.")